In [1]:
import pandas as pd
import numpy as np
import akshare as ak
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
import datetime
import math
import os
import matplotlib.pyplot as plt

In [2]:
dataset_keys = ['total_owner_equities',
    'total_assets',
    'equities_parent_company_owners',
    'total_liability',
    'inventories',
    'total_non_current_liability',
    'fixed_assets',
    'construction_materials',
    'constru_in_process',
    'total_assets',
    'total_current_assets',
    'total_current_liability',
    # income
    'operating_revenue',
    'operating_cost',
    'net_profit',
    'total_operating_revenue',
    # cash flow
    'inventory_decrease',
    'net_operate_cash_flow',
    # indicator
    'roe',
    'roa',
    'net_profit_margin',
    'gross_profit_margin',
    'inc_net_profit_year_on_year',
    'financing_expense_to_total_revenue',
    'operating_expense_to_total_revenue',
#     # rf rm
    'R_f',
    'R_m',
    # price volum
    'A股流通股本(股)',
    'A股流通市值(元)',
    '成交金额(元)',
    '总股本(股)',
    '市盈率',
    '市净率',
    '成交量(股)',
    '涨跌幅(%)',
    '换手率(%)',
    '收盘价(元)',
    # valuation
    'pcf_ratio'
    ]

In [3]:
# 读取数据
dataset_dict = {}
for key in dataset_keys : 
    dataset_dict[key] = pd.read_csv('dataset/data of factor need/' + key + '.csv', low_memory=False).set_index('date')

In [4]:
# 读取tradingdate，并获取对应一年前的交易日日期
trading_date_list = pd.read_csv('dataset/trading_date.csv').iloc[252:,:] 
dates = trading_date_list.loc[252*2:3000,'date']
def last_year_func(x):
    index_x = dates[dates.values==x].index
    try:
        return dates[index_x - 252].values[0]
    except : 
        return np.nan
dates_last = dates.apply(last_year_func)  

In [5]:
# 读取股票列表stks，这里用来自wind和来自jqdata的数据的交集
stks = list(set(dataset_dict['涨跌幅(%)'].columns).intersection(set(dataset_dict['total_assets'].columns)))
stks.sort()

In [26]:
# 获取非跨报表因子
def get_factor(dates, stks) : 
    factor_dict = {}
    factor_dict['net_profit_margin_data'] = dataset_dict['net_profit_margin'].loc[dates,stks]
    factor_dict['gross_profit_margin_data'] = dataset_dict['gross_profit_margin'].loc[dates,stks]
    factor_dict['roe_data'] = dataset_dict['roe'].loc[dates,stks]
    factor_dict['roa_data'] = dataset_dict['roa'].loc[dates,stks]
    
    # 成长因子
    factor_dict['equities_grow_data'] = (dataset_dict['total_owner_equities'].loc[dates[252:],stks].reset_index(drop=True).\
                        sub(dataset_dict['total_owner_equities'].loc[dates_last[252:],stks].reset_index(drop=True))\
                        .div(abs(dataset_dict['total_owner_equities'].loc[dates_last[252:],stks].reset_index(drop=True)))).\
                        set_index(dataset_dict['total_owner_equities'].loc[dates[252:],stks].index)
    factor_dict['asserts_grow_data'] = (dataset_dict['total_assets'].loc[dates[252:],stks].reset_index(drop=True).\
                        sub(dataset_dict['total_assets'].loc[dates_last[252:],stks].reset_index(drop=True))\
                        .div(abs(dataset_dict['total_assets'].loc[dates_last[252:],stks].reset_index(drop=True)))).\
                        set_index(dataset_dict['total_assets'].loc[dates[252:],stks].index)
    factor_dict['profit_grow_data'] = dataset_dict['inc_net_profit_year_on_year'].loc[dates,stks]
    
    net_asset_per_share_1 = dataset_dict['equities_parent_company_owners'].loc[dates[252:],stks] / dataset_dict['总股本(股)'].loc[dates[252:],stks]
    net_asset_per_share_0 = dataset_dict['equities_parent_company_owners'].loc[dates_last[252:],stks] / dataset_dict['总股本(股)'].loc[dates_last[252:],stks]
    factor_dict['NAPS_grow_data'] = (net_asset_per_share_1.reset_index(drop=True).sub(net_asset_per_share_0.reset_index(drop=True))\
                        .div(abs(net_asset_per_share_0.reset_index(drop=True)))).set_index(net_asset_per_share_1.index)
    
    ## ROE增长率（当期ROE -上期ROE）/上期ROE
    factor_dict['roe_grow_data'] = (dataset_dict['roe'].loc[dates[252:],stks].reset_index(drop=True).\
                        sub(dataset_dict['roe'].loc[dates_last[252:],stks].reset_index(drop=True))\
                        .div(abs(dataset_dict['roe'].loc[dates_last[252:],stks].reset_index(drop=True)))).\
                        set_index(dataset_dict['roe'].loc[dates[252:],stks].index)
    ## 主营业务收入增长率 （当期主营业务收入 -上期主营业务收入）/上期主营业务收入   
    factor_dict['operating_revenue_grow_data'] = (dataset_dict['operating_revenue'].loc[dates[252:],stks].reset_index(drop=True).\
                        sub(dataset_dict['operating_revenue'].loc[dates_last[252:],stks].reset_index(drop=True))\
                        .div(abs(dataset_dict['operating_revenue'].loc[dates_last[252:],stks].reset_index(drop=True)))).\
                        set_index(dataset_dict['operating_revenue'].loc[dates[252:],stks].index)

    # 杠杆因子
    ## 流通股本/总股本
    factor_dict['shares_cir_pct_data'] = dataset_dict['A股流通股本(股)'].loc[dates[252:],stks] / dataset_dict['总股本(股)'].loc[dates[252:],stks]
    factor_dict['liab_to_assests_data'] = dataset_dict['total_liability'].loc[dates[252:],stks] / dataset_dict['total_assets'].loc[dates[252:],stks]
    # 估值因子
    ## CFP 经营现金流/总市值
    factor_dict['cfp_data'] = 1 / dataset_dict['pcf_ratio'].loc[dates[252:],stks]
    ## EP PE的倒数
    factor_dict['ep_data'] = 1 / dataset_dict['市盈率'].loc[dates[252:],stks]
    ## BP PB的倒数
    factor_dict['bp_data'] = 1 / dataset_dict['市净率'].loc[dates[252:],stks]
    ## 1个月成交金额
    def apply_func(date):
        return dataset_dict['成交金额(元)'][dataset_dict['成交金额(元)'].index <= date].sort_index().tail(21).sum()
    factor_dict['money_one_month_data'] = dates[252:].apply(apply_func).set_index(dates[252:])
    ## 近三个月平均成交量
    def apply_func(date):
        return dataset_dict['成交量(股)'][dataset_dict['成交量(股)'].index <= date].sort_index().tail(21*3).sum()
    factor_dict['volume_three_month_data'] = dates[252:].apply(apply_func).set_index(dates[252:])
    ## 换手率
    factor_dict['turnover_rate_data'] = dataset_dict['换手率(%)'].loc[dates,stks]
    # 技术因子
    ## 一个月股价反转 （当前股价 - 一个月前股价）/一个月前股价
    def one_months_ago(x):
        index_x = dates[dates.values==x].index
        try:
            return dates[index_x - 21].values[0]
        except : 
            return np.nan
    def three_months_ago(x):
        index_x = dates[dates.values==x].index
        try:
            return dates[index_x - 21 * 3].values[0]
        except : 
            return np.nan
    def six_months_ago(x):
        index_x = dates[dates.values==x].index
        try:
            return dates[index_x - 21 * 6].values[0]
        except : 
            return np.nan

    dates_i = dates.apply(one_months_ago)
    factor_dict['price_reverses_1'] = (dataset_dict['收盘价(元)'].loc[dates[252:],stks].reset_index(drop=True).\
                        sub(dataset_dict['收盘价(元)'].loc[dates_i[252:],stks].reset_index(drop=True))\
                        .div(abs(dataset_dict['收盘价(元)'].loc[dates_i[252:],stks].reset_index(drop=True)))).\
                        set_index(dataset_dict['收盘价(元)'].loc[dates[252:],stks].index)        
    ## 三个月股价反转 （当前股价 - 三个月前股价）/三个月前股价
    dates_i = dates.apply(three_months_ago)
    factor_dict['price_reverses_3'] = (dataset_dict['收盘价(元)'].loc[dates[252:],stks].reset_index(drop=True).\
                        sub(dataset_dict['收盘价(元)'].loc[dates_i[252:],stks].reset_index(drop=True))\
                        .div(abs(dataset_dict['收盘价(元)'].loc[dates_i[252:],stks].reset_index(drop=True)))).\
                        set_index(dataset_dict['收盘价(元)'].loc[dates[252:],stks].index)        
    ## 六个月股价反转 （当前股价 - 六个月前股价）/六个月前股价
    dates_i = dates.apply(six_months_ago)
    factor_dict['price_reverses_6'] = (dataset_dict['收盘价(元)'].loc[dates[252:],stks].reset_index(drop=True).\
                        sub(dataset_dict['收盘价(元)'].loc[dates_i[252:],stks].reset_index(drop=True))\
                        .div(abs(dataset_dict['收盘价(元)'].loc[dates_i[252:],stks].reset_index(drop=True)))).\
                        set_index(dataset_dict['收盘价(元)'].loc[dates[252:],stks].index)     
    dates.index = dates
    ## 最高价长度 前20日内最高价距离现在的天数 这里取前20个交易日
    def apply_func(date): 
        close_20 = dataset_dict['收盘价(元)'][dataset_dict['收盘价(元)'].index <= date][stks].sort_index().tail(20)
        close_20.index = pd.to_datetime(close_20.index,format='%Y-%m-%d')
        tt = (datetime.datetime.strptime(date, '%Y-%m-%d') - close_20.idxmax())#
        return tt.dt.days.rename(date)
    factor_dict['max_price_length_data'] = dates[252:].apply(apply_func)
    
    ## 容量比     前26日内上涨的成交量/前26日内下降的成交量    这里取前26个交易日
    def apply_func(date):
        volume = dataset_dict['成交量(股)'][dataset_dict['成交量(股)'].index < date].sort_index().tail(26)        
        chg = dataset_dict['涨跌幅(%)'][dataset_dict['涨跌幅(%)'].index < date].sort_index().tail(26)        
        series_26 = volume[chg>0].sum() / volume[chg<0].sum()
        volume_ratio_data = pd.Series(series_26,index=stks)
        return volume_ratio_data
    factor_dict['volume_ratio_data'] = dates[252:].apply(apply_func)    # 规模因子
    ## 流通市值  自由流通市值
    factor_dict['market_value_cir_data'] = dataset_dict['A股流通市值(元)'].loc[dates[252:],stks]
    ## 总资产
    factor_dict['total_assets_data'] = dataset_dict['total_assets'].loc[dates[252:],stks]

    
    # 质量因子
    ## 存货周转率   营业成本／[(期初存货净额+期末存货净额)／2]
    ## 长期负债比率  长期负债总额／负债合计 或  (长期借款+应付债券+isnull((长期应付款+专项应付款),长期应付款合计）／负债合计*100%
    factor_dict['long_liab_rate_data'] = dataset_dict['total_non_current_liability'].loc[dates[252:],stks] / dataset_dict['total_liability'].loc[dates[252:],stks]
    ## 每股负债比 负债总额 /股东权益
    factor_dict['debt_ratio_per_share_data'] = dataset_dict['total_liability'].loc[dates[252:],stks] / dataset_dict['total_owner_equities'].loc[dates[252:],stks]
    ## 固定比 （固定资产+工程物资+在建工程）/总资产
    factor_dict['fix_ratio_data'] = (dataset_dict['fixed_assets'].loc[dates[252:],stks].fillna(0) + \
                      dataset_dict['construction_materials'].loc[dates[252:],stks].fillna(0) + \
                     dataset_dict['constru_in_process'].loc[dates[252:],stks].fillna(0)) / dataset_dict['fixed_assets'].loc[dates[252:],stks]
    ## 速动比率  速动比率 = (流动资产 - 存货) / 流动负债
    factor_dict['quick_ratio_data'] = (dataset_dict['total_current_assets'].loc[dates[252:],stks].fillna(0) + \
                      dataset_dict['inventories'].loc[dates[252:],stks].fillna(0)) / dataset_dict['total_current_liability'].loc[dates[252:],stks]
    ## 流动比率  流动资产合计/流动负债合计
    factor_dict['current_ratio_data'] = dataset_dict['total_current_assets'].loc[dates[252:],stks] / dataset_dict['total_current_liability'].loc[dates[252:],stks]
    ## 净利润现金占比  经营现金流量净额 /净利润
    ## 总资产周转率  营业总收入／[(期初资产总额+期末资产总额)／2]
    ## 流动负债率  流动负债总额/总负债
    factor_dict['liqdr_data'] = dataset_dict['total_current_liability'].loc[dates[252:],stks] / dataset_dict['total_liability'].loc[dates[252:],stks]
    ## 财务费用比例  财务费用/营业收入
    factor_dict['financing_expense_ratio_data'] = dataset_dict['financing_expense_to_total_revenue'].loc[dates[252:],stks]
    ## 营业费用比例  营业费用/营业收入
    factor_dict['operating_expense_ratio_data'] = dataset_dict['operating_expense_to_total_revenue'].loc[dates[252:],stks]
    return factor_dict
    
factor_dict = get_factor(dates, stks)    
for key in factor_dict.keys(): 
    factor_dict[key].to_csv('dataset/36factor/' + key + '.csv')

In [ ]:
## 三个跨报表因子，使用for循环重复计算

In [ ]:
## 净利润现金占比  经营现金流量净额 /净利润
def net_profit_cash_ratio(stks, dates) : 
    result = pd.DataFrame(index=dates, columns=stks)
    for stk in stks : 
        print(stk)
        cf_data = pd.read_csv('dataset/A股各公司财务数据/合并现金流量表/' + stk[:6] + '.csv', encoding='gbk')\
                    [['pub_date','start_date','end_date','report_date','report_type','source_id','source','net_operate_cash_flow']]
        income_data = pd.read_csv('dataset/A股各公司财务数据/合并利润表/' + stk[:6] + '.csv', encoding='gbk')\
                    [['pub_date','start_date','end_date','report_date','report_type','source_id','source','net_profit']]
        merge_data=pd.merge(cf_data,income_data,on=['start_date','end_date','report_date','report_type','source_id','source'], 
                            how='inner')
        merge_data['pub_date'] = merge_data[['pub_date_x','pub_date_y']].max(axis=1)
        merge_data = merge_data.sort_values(by = ['pub_date','end_date','start_date']).reset_index(drop=True) 
        if len(merge_data) >= 2 : 
            for i in merge_data.index[:-1]:  
                if merge_data.loc[i, 'pub_date'] == merge_data.loc[i+1, 'pub_date']:
                    merge_data.drop(index = i,inplace = True)
        merge_data = merge_data.set_index('pub_date')
        new_df = pd.DataFrame(index=dates)
        merge_data = pd.merge(new_df,merge_data, how='left', left_index=True, right_index=True).sort_index() # 合并交易日历和数据
        merge_data.fillna(method='pad', inplace=True) # ffill
        
        result[stk] = merge_data['net_operate_cash_flow'] / merge_data['net_profit']
    return result

factor_dict = {}    
key = 'net_profit_cash_ratio_data'
factor_dict[key] = net_profit_cash_ratio(stks, dates[252:])
factor_dict[key].to_csv('dataset/36factor/' + key + '.csv')

In [ ]:
# 营业收入／期末总资产
def asset_turnover(stks, date) : 
    result = pd.DataFrame(index=date, columns=stks)

    for stk in stks : 
        print(stk)
        balance_data = pd.read_csv('dataset/A股各公司财务数据/合并资产负债表/' + stk[:6] + '.csv', encoding='gbk')\
                    [['pub_date','start_date','end_date','report_date','report_type','source_id','source','total_assets']]\
                    .sort_values(by=['pub_date','end_date'])
        income_data = pd.read_csv('dataset/A股各公司财务数据/合并利润表/' + stk[:6] + '.csv', encoding='gbk')\
                    [['pub_date','start_date','end_date','report_date','report_type','source_id','source','operating_revenue']]
        merge_data=pd.merge(balance_data,income_data,on=['start_date','end_date','report_date','report_type','source_id','source'], 
                            how='inner')
        merge_data['pub_date'] = merge_data[['pub_date_x','pub_date_y']].max(axis=1)
        merge_data = merge_data.sort_values(by = ['pub_date','end_date','start_date']).reset_index(drop=True) 
        if len(merge_data) >= 2 : 
            for i in merge_data.index[:-1]:  
                if merge_data.loc[i, 'pub_date'] == merge_data.loc[i+1, 'pub_date']:
                    merge_data.drop(index = i,inplace = True)
        merge_data = merge_data.set_index('pub_date')
        new_df = pd.DataFrame(index=dates)
        merge_data = pd.merge(new_df,merge_data, how='left', left_index=True, right_index=True).sort_index() # 合并交易日历和数据
        merge_data.fillna(method='pad', inplace=True) # ffill
        
        result[stk] = merge_data['operating_revenue'] / merge_data['total_assets']
    return result
factor_dict = {} 
key = 'asset_turnover_data'
factor_dict[key] = asset_turnover(stks, dates)
factor_dict[key].to_csv('dataset/36factor/' + key + '.csv')

In [ ]:
# 营业成本／期末存货净额
def inventory_turnover(stks, dates) : 
    result = pd.DataFrame(index=dates, columns=stks)

    for stk in stks : 
        print(stk)
        balance_data = pd.read_csv('dataset/A股各公司财务数据/合并资产负债表/' + stk[:6] + '.csv', encoding='gbk')\
                    [['pub_date','start_date','end_date','report_date','report_type','source_id','source','inventories']]\
                    .sort_values(by=['pub_date','end_date'])
        income_data = pd.read_csv('dataset/A股各公司财务数据/合并利润表/' + stk[:6] + '.csv', encoding='gbk')\
                    [['pub_date','start_date','end_date','report_date','report_type','source_id','source','operating_cost']]
        merge_data=pd.merge(balance_data,income_data,on=['start_date','end_date','report_date','report_type','source_id','source'], 
                            how='inner')
        merge_data['pub_date'] = merge_data[['pub_date_x','pub_date_y']].max(axis=1)
        merge_data = merge_data.sort_values(by = ['pub_date','end_date','start_date']).reset_index(drop=True) 
        if len(merge_data) >= 2 : 
            for i in merge_data.index[:-1]:  
                if merge_data.loc[i, 'pub_date'] == merge_data.loc[i+1, 'pub_date']:
                    merge_data.drop(index = i,inplace = True)
        merge_data = merge_data.set_index('pub_date')
        new_df = pd.DataFrame(index=dates)
        merge_data = pd.merge(new_df,merge_data, how='left', left_index=True, right_index=True).sort_index() # 合并交易日历和数据
        merge_data.fillna(method='pad', inplace=True) # ffill
        
        result[stk] = merge_data['operating_cost'] / merge_data['inventories']
    return result
factor_dict = {}
key = 'inventory_turnover_data'
factor_dict[key] = inventory_turnover(stks, dates)
factor_dict[key].to_csv('dataset/36factor/' + key + '.csv')